In [3]:
# Precise feature mapping
FEATURE_MAPPING = {
    # Personal info
    'name': {'sheet': 'person_202510150920', 'column': 'name'},   
    'sex': {'sheet': 'person_202510150920', 'column': 'sex'},
    'nation': {'sheet': 'person_202510150920', 'column': 'nation'},
    'region': {'sheet': 'person_202510150920', 'column': 'region'},
    'priority': {'sheet': 'person_202510150920', 'column': 'priority'},
    
    # Student program info
    'student_code': {'sheet': 'student_202510150856', 'column': 'student_code'},
    'program_code': {'sheet': 'student_202510150856', 'column': 'program_code'},
    'program_name': {'sheet': 'student_202510150856', 'column': 'program_name'},
    'major_code': {'sheet': 'student_202510150856', 'column': 'major_code'},
    'major_name': {'sheet': 'student_202510150856', 'column': 'major_name'},
    'class_code': {'sheet': 'student_202510150856', 'column': 'class_code'},
    'class_name': {'sheet': 'student_202510150856', 'column': 'class_name'},
    
    # Admission info
    'admission_year': {'sheet': 'candidate_202510150926', 'column': 'admission_year'},
    'admission_order': {'sheet': 'candidate_202510150926', 'column': 'admission_order'},
    'admission_code': {'sheet': 'candidate_202510150926', 'column': 'admission_code'},
    'admission_combination_code': {'sheet': 'candidate_202510150926', 'column': 'admission_combination_code'},
    'admission_score': {'sheet': 'candidate_202510150926', 'column': 'admission_score'},
    'encourage_point': {'sheet': 'candidate_202510150926', 'column': 'encourage_point'},
    'subject_point_1': {'sheet': 'candidate_202510150926', 'column': 'subject_point_1'},
    'subject_point_2': {'sheet': 'candidate_202510150926', 'column': 'subject_point_2'},
    'subject_point_3': {'sheet': 'candidate_202510150926', 'column': 'subject_point_3'},
    
    # Person details
    'avg_12': {'sheet': 'person_detail_202510150920', 'column': 'avg_12'},
    'tt_province': {'sheet': 'person_detail_202510150920', 'column': 'tt_province'},
    'tt_district': {'sheet': 'person_detail_202510150920', 'column': 'tt_district'},
    'province_12_code': {'sheet': 'person_detail_202510150920', 'column': 'province_12_code'},
    'school_12_code': {'sheet': 'person_detail_202510150920', 'column': 'school_12_code'},
    
    # Primary key
    'person_code_hashed': {'sheet': 'student_202510150856', 'column': 'person_code_hashed'},
}

print("✅ Feature mapping loaded")
print(f"Total features: {len(FEATURE_MAPPING)}")


✅ Feature mapping loaded
Total features: 27


In [ ]:
import pandas as pd
import numpy as np
import os

# Input and output paths
file_path = 'base_data/student_hashed.xlsx'
output_xlsx = 'data_prepared/student_infor.xlsx'
output_csv = 'data_prepared/student_infor.csv'

# Create output directory
os.makedirs('data_prepared', exist_ok=True)

print("🔄 MEMORY-OPTIMIZED CHUNK PROCESSING (FIXED)")
print("💾 Max RAM: ~500MB | Time: 3-7 min")

# Feature groups by sheet
feature_groups = {
    'person_202510150920': ['name', 'sex', 'nation', 'region', 'priority'],
    'student_202510150856': ['student_code', 'program_code', 'program_name', 'major_code', 'major_name', 'class_code', 'class_name'],
    'candidate_202510150926': ['admission_year', 'admission_order', 'admission_code', 'admission_combination_code', 'admission_score', 'encourage_point', 'subject_point_1', 'subject_point_2', 'subject_point_3'],
    'person_detail_202510150920': ['avg_12', 'tt_province', 'tt_district', 'province_12_code', 'school_12_code'],
}

CHUNK_SIZE = 50000
EXCEL_ROW_LIMIT = 1048576

print("📊 LOADING REFERENCE KEYS...")
xl = pd.ExcelFile(file_path)
student_df = pd.read_excel(xl, 'student_202510150856', usecols=['person_code_hashed'])
student_df['person_code_hashed'] = student_df['person_code_hashed'].astype(str).str.strip()
keys = student_df[student_df['person_code_hashed'].notna() & (student_df['person_code_hashed'] != '')]['person_code_hashed'].unique()
print(f"✅ {len(keys):,} unique person codes")

# Step 2: Build lookup dictionaries with MANUAL CHUNKING
print("\n🔑 BUILDING LOOKUP DICTIONARIES...")
lookups = {}

def read_sheet_chunks(xl, sheet, cols, keys_set, chunk_size=CHUNK_SIZE):
    """Manual chunking using skiprows + nrows"""
    lookup = {}
    total_rows = 0
    
    # Get total rows in sheet (first read with nrows=1 to get shape)
    try:
        test_df = pd.read_excel(xl, sheet_name=sheet, nrows=1, usecols=cols)
        # Estimate total rows by reading until empty
        row_count = 0
        while True:
            chunk = pd.read_excel(xl, sheet_name=sheet, skiprows=row_count, nrows=chunk_size, usecols=cols)
            if chunk.empty:
                break
            row_count += len(chunk)
        total_rows = row_count
    except:
        total_rows = len(keys) * 2  # Fallback
    
    print(f"     Total rows: {total_rows:,}")
    
    # Process chunks
    for start_row in range(0, total_rows, chunk_size):
        chunk = pd.read_excel(xl, sheet_name=sheet, skiprows=start_row, nrows=chunk_size, usecols=cols)
        
        if chunk.empty:
            break
            
        # Clean key
        chunk['person_code_hashed'] = chunk['person_code_hashed'].astype(str).str.strip()
        chunk = chunk[chunk['person_code_hashed'].isin(keys_set)]
        
        # Build lookup
        for _, row in chunk.iterrows():
            key = row['person_code_hashed']
            if key not in lookup:
                lookup[key] = {}
            for col in cols:
                if col != 'person_code_hashed':
                    lookup[key][col] = row[col]
        
        if (start_row // chunk_size) % 10 == 0:
            print(f"     Processed {start_row:,}/{total_rows:,} rows...", end='\r')
    
    return lookup

# Add key column to each group
for sheet, cols in feature_groups.items():
    cols.append('person_code_hashed')

keys_set = set(keys)  # Faster lookup

for sheet, cols in feature_groups.items():
    print(f"   Processing sheet: {sheet}")
    lookups[sheet] = read_sheet_chunks(xl, sheet, cols, keys_set)
    print(f"✅ {sheet}: {len(lookups[sheet]):,} records")

# Step 3: MERGE using dictionaries
print("\n🔗 MERGING RECORDS...")
final_data = []
feature_order = [
    'name', 'sex', 'nation', 'region', 'priority',
    'student_code', 'program_code', 'program_name', 'major_code', 'major_name', 'class_code', 'class_name',
    'admission_year', 'admission_order', 'admission_code', 'admission_combination_code', 'admission_score', 
    'encourage_point', 'subject_point_1', 'subject_point_2', 'subject_point_3',
    'avg_12', 'tt_province', 'tt_district', 'province_12_code', 'school_12_code',
    'person_code_hashed'
]

total_complete = 0
for i, key in enumerate(keys):
    if i % 100000 == 0:
        print(f"   Merging {i:,}/{len(keys):,} ({i/len(keys)*100:.1f}%)", end='\r')
    
    # Check if ALL sheets have this key
    record_complete = True
    merged_row = {'person_code_hashed': key}
    
    for sheet in feature_groups.keys():
        if key in lookups[sheet]:
            merged_row.update(lookups[sheet][key])
        else:
            record_complete = False
            break
    
    if record_complete:
        ordered_row = {k: merged_row.get(k, None) for k in feature_order}
        final_data.append(ordered_row)
        total_complete += 1

print(f"\n✅ Complete records: {total_complete:,}/{len(keys):,} ({total_complete/len(keys)*100:.1f}%)")

# Step 4: Save
print("\n💾 SAVING FILES...")
df_final = pd.DataFrame(final_data)

# Save CSV
df_final.to_csv(output_csv, index=False)
print(f"✅ CSV saved: {len(df_final):,} rows")

# Save XLSX if fits
if len(df_final) <= EXCEL_ROW_LIMIT:
    df_final.to_excel(output_xlsx, index=False, engine='openpyxl')
    print(f"✅ XLSX saved")
else:
    print(f"⚠️  XLSX skipped: {len(df_final):,} > {EXCEL_ROW_LIMIT:,} rows")

print("\n" + "="*60)
print("🎉 SUCCESS! CHUNK PROCESSING COMPLETE!")
print(f"📊 Final: {len(df_final):,} students × {len(feature_order)} features")
print(f"📁 Files: {output_csv}")
if os.path.exists(output_xlsx):
    print(f"📁 Files: {output_xlsx}")
print("="*60)

🔄 MEMORY-OPTIMIZED CHUNK PROCESSING (FIXED)
💾 Max RAM: ~500MB | Time: 3-7 min
📊 LOADING REFERENCE KEYS...
✅ 18,876 unique person codes

🔑 BUILDING LOOKUP DICTIONARIES...
   Processing sheet: person_202510150920
     Total rows: 37,752
✅ person_202510150920: 18,864 records
   Processing sheet: student_202510150856
     Total rows: 37,752
✅ student_202510150856: 18,876 records
   Processing sheet: candidate_202510150926
     Total rows: 37,752
✅ candidate_202510150926: 16,580 records
   Processing sheet: person_detail_202510150920
     Total rows: 37,752
✅ person_detail_202510150920: 16,656 records

🔗 MERGING RECORDS...
   Merging 0/18,876 (0.0%)
✅ Complete records: 16,580/18,876 (87.8%)

💾 SAVING FILES...
✅ CSV saved: 16,580 rows
✅ XLSX saved

🎉 SUCCESS! CHUNK PROCESSING COMPLETE!
📊 Final: 16,580 students × 27 features
📁 Files: data_prepared/student_infor.csv
📁 Files: data_prepared/student_infor.xlsx
